In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

ds = load_dataset("kdave/Indian_Financial_News")

README.md:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

training_data_26000.csv:   0%|          | 0.00/115M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26961 [00:00<?, ? examples/s]

In [ ]:
import pandas

df = ds['train'].to_pandas()
df

,URL,Content,Summary,Sentiment
0,https://www.moneycontrol.com/news/business/eco...,US consumer spending dropped by a record in Ap...,consumer spending plunges 13.6 percent in Apri...,Negative
1,https://www.businesstoday.in/top-story/state-r...,State-run lenders require an urgent Rs 1.2 tri...,government will have to take a bulk of the tab...,Negative
2,https://www.financialexpress.com/economy/covid...,Apparel exporters on Wednesday urged the gover...,exporters are facing issues in terms of raw ma...,Negative
3,https://www.moneycontrol.com/news/business/mar...,Asian shares battled to extend a global reboun...,the dollar loses some ground on the safe haven...,Negative
4,https://www.financialexpress.com/industry/six-...,After India’s sovereign credit rating fell to ...,six Indian public-sector undertakings have tak...,Negative
...,...,...,...,...
26956,https://www.financialexpress.com/economy/amit-...,Union Home Minister Amit Shah on Saturday comp...,home minister congratulates prime minister for...,Positive
26957,https://economictimes.indiatimes.com/news/poli...,Saab Bags India’s First 100% FDI in Defence Pr...,first 100% foreign direct investment in defenc...,Positive
26958,https://www.moneycontrol.com/news/business/com...,"Crude oil futures gained to Rs 2,136 per barre...",crude oil futures gain 168.67 percent from low...,Positive
26959,https://economictimes.indiatimes.com/markets/s...,Bloomberg\n\nEmerging markets struggling with ...,"the u.s. economy is powering ahead, adding ove...",Positive


In [ ]:
from transformers import Trainer, TrainingArguments
df2 = df.copy()

In [ ]:
from sklearn.model_selection import train_test_split
df.columns

y = df2['Sentiment']
X = df2.drop(['URL','Summary','Sentiment'], axis=1)

In [ ]:
X_train_temp, X_temp, y_train_temp, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Split temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    """
    Compute metrics for evaluation during training.

    Args:
        pred: A PredictionOutput object containing predictions and labels.

    Returns:
        dict: Dictionary of computed metrics (accuracy, precision, recall, F1 score).
    """
    # Extract predictions and labels
    predictions = pred.predictions.argmax(-1)  # Take the index of the highest logit
    labels = pred.label_ids

    # Calculate metrics
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [ ]:
from datasets import load_dataset, Dataset
def preprocess_function(examples):
    # Define a max_length for truncation and padding
    max_length = 512  # Or any other appropriate length for your model
    return tokenizer(examples["Content"], padding="max_length", truncation=True, max_length=max_length)

# Add Sentiment as a column in the datasets before preprocessing
train_dataset = Dataset.from_pandas(X_train_temp.assign(Sentiment=y_train_temp))
eval_dataset = Dataset.from_pandas(X_val.assign(Sentiment=y_val))

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Rename 'Sentiment' to 'labels'
train_dataset = train_dataset.rename_column("Sentiment", "labels")
eval_dataset = eval_dataset.rename_column("Sentiment", "labels")

training_args = TrainingArguments(
    output_dir='./results',              # Directory to save the model and logs
    num_train_epochs=3,                  # Number of training epochs
    per_device_train_batch_size=8,       # Batch size for training
    per_device_eval_batch_size=16,       # Batch size for evaluation
    warmup_steps=500,                    # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                   # Weight decay to apply (if any)
    logging_dir='./logs',                # Directory to save logs
    logging_steps=10,                    # How often to log training info
    evaluation_strategy="epoch",         # Evaluate model at the end of each epoch
    save_strategy="epoch",               # Save model at the end of each epoch
    load_best_model_at_end=True,         # Load the best model when finished training
    metric_for_best_model="accuracy",   # Metric to track when loading the best model
    greater_is_better=True,              # Whether a higher metric is better (for accuracy)
    lr_scheduler_type="linear",          # Linear learning rate scheduler
    learning_rate=2e-5,                  # Initial learning rate
    logging_first_step=True,             # Log the first step
)
# Now, proceed with the Trainer setup
trainer = Trainer(
    model=finbert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


Map:   0%|          | 0/18872 [00:00<?, ? examples/s]

Map:   0%|          | 0/4044 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.